##  A complex model in JuMP: Biomass feedstock optimization
The factory example above is a pretty simple problem. Useful for introducing the basic anatomy of a JuMP model, but hardly worth relying on algebraic programming to find the solution (as it can be obtained analytically without much trouble). So let's now turn to a more complex example...


Assume you are the operator of a biorefinery with a fixed capacity $K$ and you are looking to optimize the types and quantity of feedstock you can purchase to maximize the operating profits for your facility over multiple time periods (e.g. months, years)

Given the time-dependent availability of different biomass feedstocks and time-dependent price of fuel products as well as fixed biofuel production and biomass storage capacity, you need to decide the feed purchase decisions that will maximize your profit. 

Sounds like a good optimization problem!

### 1. Load data and define parameters

Before we get started defining our model, we need define a set of **parameters** or constant values used in our optimization problem.

Parameters that are constant over time:

- Biofuel production capacity: $K = 20 \times 10^5$ tonnes/years
- Feedstock cost - dedicated energy crops, wood chips: $\mathbf{c} = [30,50] $ $/tonne
- Liquid fuel yield - dedicated energy crops, wood chips: $\mathbf{e} = [12, 20] $ GJ/tonne
- Available storage capacity - dedicated energy crops, wood chips: $\mathbf{H} = [30, 15] $ tonnes
- Variable cost of storage - dedicated energy crops, wood chips: $\mathbf{s} = [1,1.5] $ $/tonne
- Plant minimum energy production rate: $E^{min} = 200 $ GJ/year
- Initial inventory for storage: $\mathbf{\bar{I}_0} = 0 $ tonnes


We also need to load some time-dependent parameters, i.e whose values vary across the time periods (years) of interest for the problem, specifically, the supply of each feedstock in each period and the price of fuel in each period.



In [1]:
# Before you use a package in Julia for the first time, you must "add" (install) it. 
# We have included a project file that specifies the packages we need.
# You can install the packages specified in the project file by following these commands
# Make sure you have set the kernel to Julia in your Jupyter notebook
import Pkg
Pkg.activate(".")
Pkg.instantiate()
using Plots
using DataFrames
using HiGHS
using JuMP
using Plots


  Activating project at `c:\Users\dm6004\Dropbox (MIT)\NYU_teaching\EnergySystems_course\Codes\optimization_intro`


We see that supply of dedicated energy crops is greater than supply of wood chips, which are the most expensive feedstock.  The yearly variations in biomass availability can be attributed to weather-driven sensitivity to crop yields.

Lets now load the above data as well as time-dependent data

In [3]:
using DataFrames
using CSV

# Load input parameters 
inputs_constant = DataFrame(CSV.File("input_data.csv"))

pFeedCost = Array(inputs_constant[:,2])  # feedstock cost data as Array
pFuelYield = Array(inputs_constant[:,3])  # fuel yield from each feedstock
pStorageCap = Array(inputs_constant[:,4])  # storage capacity data in 10^5 tonnes
pStorageCost = Array(inputs_constant[:,5])  # storage operating cost

pFacilityCap = 20  # facility capacity in 10^5 tonnes per year
pInitialStorage = [0, 0]  # initial storage inventory in 10^5 tonnes
pFacilityMinInput = 0.4 * pFacilityCap  # minimum facility input in 10^5 tonnes per year
# Load time series data
inputs = DataFrame(CSV.File("input_data_time_series.csv"))

T = 1: length(inputs[:,1]) # Number of time steps
pSupply = Matrix(inputs[:,2:3])  # biomass supply data as Array in 10^5 tonnes per year
pFuelPrice = Array(inputs[:,4])  # fuel price data as Array in $/GJ
C = 1:length(pSupply[1,:])  # set of biomass types

println("Data loaded successfully") 
println("Number of time steps: ", length(T))
println("Supply data: ")
println(pSupply)
println("Fuel price data: ")
println(pFuelPrice)
println("Feedstock cost data: ")
println(pFeedCost)
println("Fuel yield data: ")
println(pFuelYield)
println("Storage capacity data: ")
println(pStorageCap)
println("Storage operating cost data: ")
println(pStorageCost)
println("Facility capacity: ", pFacilityCap)
println("Data loaded successfully")

Data loaded successfully
Number of time steps: 3
Supply data: 
[20 5; 10 12; 15 10]
Fuel price data: 
[3.0, 2.6, 3.2]
Feedstock cost data: 
[30, 50]
Fuel yield data: 
[12, 20]
Storage capacity data: 
[30, 15]
Storage operating cost data: 
[1.0, 1.5]
Facility capacity: 20
Data loaded successfully


### 2. Define the model
Now that we have our parameters set, let's define a new optimization model, a linear programming problem where we will use HIGHs as the optimizer


In [4]:
bio_model = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

### 3. Model variables ###

The next step would be to define the model decision variables that need to be optimized.

- Consumption of biomass feed of type c in period t: $vCONS_{c,t} \quad \forall c \in C, t \in T$ 
- Purchase of biomass feed of type c in period t: $vPUR_{c,t} \quad \forall c \in C, t \in T$ 

- Inventory of biomass feed of type c in storage at the end of period t: $vSTOR_{c,t} \quad \forall c \in C, t \in T$ 

Where:
$c \in C$: Set of biomass feedstock types - dedicated energy crops, wood chips
$t \in T$: Set of time periods (years) - [1,2,3]

This is our first example of creating an array of variables and **indexing across sets**. In practice, you'll frequently define arrays of variables and constraints that repeat across various sets or dimensions of your data (e.g. time periods, generators, regions, transmission lines). Note that variables can be indexed across multiple dimensions in N-dimensional containers (although here we will only index across the single dimension: time periods). [Read more on variable containers here](https://jump.dev/JuMP.jl/stable/manual/variables/#Variable-containers).

Additionally, we will define bounds for each decision variable based on the parameters set previously. We'll set production rate upper bounds later, as these vary by time interval depending on the availability of biomass in each time period.

Note: by convention, and to make it easy to spot a decision variable in future expressions, we'll use the convention that all decision variable names will be preceeded by symbol 'v'. (This is an optional convention, and you can feel free to adopt your own, but we recommend adopting a consistent convention for naming variables, constraints, expressions, parameters, etc.)






In [5]:
# Decision variables and bounds:

@variables(bio_model, begin
                          vCONS[c in C, t in T]      >= 0
                          vPUR[c in C, t in T]      >= 0
                          vSTOR[c in C, t in T]     >= 0
        end)
print(bio_model)

Feasibility
Subject to
 vCONS[1,1] >= 0
 vCONS[2,1] >= 0
 vCONS[1,2] >= 0
 vCONS[2,2] >= 0
 vCONS[1,3] >= 0
 vCONS[2,3] >= 0
 vPUR[1,1] >= 0
 vPUR[2,1] >= 0
 vPUR[1,2] >= 0
 vPUR[2,2] >= 0
 vPUR[1,3] >= 0
 vPUR[2,3] >= 0
 vSTOR[1,1] >= 0
 vSTOR[2,1] >= 0
 vSTOR[1,2] >= 0
 vSTOR[2,2] >= 0
 vSTOR[1,3] >= 0
 vSTOR[2,3] >= 0


### 4. Define Expressions 

Now we define an expressions that calculate the revenues and operating cost for each time period over the problem time horizon.

Note that by convention, and to distinguish expression names from decision variable names and constraint names, we will use the convention that all expression names take the format `eExpressionName`. (This is an optional convention, and you can feel free to adopt your own, but we recommend adopting a consistent convention for naming variables, constraints, expressions, parameters, etc.)

1. We will define the operating revenues as:
$$ 
eREV_t =\sum_{c\in C} {p_t \times e_c \times vCONS_{c,t}} \quad \forall t \in T
$$

2. We will define the operating costs as:
$$ 
eOPEX_t =\sum_{c\in C} {(c_c \times vPUR_{c,t}+s_c \times vSTOR_{c,t})} \quad \forall \in T
$$

As with our variables, we will define this expression as an array indexed across the set of time periods `t` in the time series `T` and over the set of feedstocks `C`. Here we see our first example of the syntax for a sum expression.


In [6]:
unregister(bio_model, :eREV)
unregister(bio_model, :eOPEX)
# Expressions defining system revenues and costs
@expression(bio_model, eREV[t in T], 
    sum( (pFuelPrice[t] * pFuelYield[c] * vCONS[c,t]) for c in C) );

@expression(bio_model, eOPEX[t in T], 
    sum( (pFeedCost[c] * vPUR[c,t] + pStorageCost[c] * vSTOR[c,t]) for c in C) );

In [7]:

# Show the Expressions
@show(eREV[1:end]);
@show(eOPEX[1:end]);

eREV[1:end] = 1-dimensional DenseAxisArray{AffExpr,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{AffExpr}:
 36 vCONS[1,1] + 60 vCONS[2,1]
 31.200000000000003 vCONS[1,2] + 52 vCONS[2,2]
 38.400000000000006 vCONS[1,3] + 64 vCONS[2,3]
eOPEX[1:end] = 1-dimensional DenseAxisArray{AffExpr,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{AffExpr}:
 30 vPUR[1,1] + vSTOR[1,1] + 50 vPUR[2,1] + 1.5 vSTOR[2,1]
 30 vPUR[1,2] + vSTOR[1,2] + 50 vPUR[2,2] + 1.5 vSTOR[2,2]
 30 vPUR[1,3] + vSTOR[1,3] + 50 vPUR[2,3] + 1.5 vSTOR[2,3]


Note that JuMP automatically interprets our expression definitions by converting the parameter references to constants and distributing terms into a [linear combination](https://en.wikipedia.org/wiki/Linear_combination) of coefficients and decision variabes, as per the displayed text above. So feel free to define these expressions however you'd like, and JuMP will take care of the math...

### 5. Constraints

Now we define a set of constraints that define the feasible region for our decision variables, based on the engineering constraints of our biomass conversion and storage system.

Note that by convention, and to distinguish constraint names from decision variable names and expression names, we will use the convention that all constraint names take the format `cConstraintName`. (This is an optional convention, and you can feel free to adopt your own, but we recommend adopting a consistent convention for naming variables, constraints, expressions, parameters, etc.)

1. First, the biomass conversion capacity in each period is fixed as per the facility throughput capacity. 

$$
\sum_{c \in C}vCONS_{c,t} \leq  pFacilityCap \quad \forall t \in T
$$

2. Similarly, the storage capacity for each biomass type is fixed.
$$
vSTOR_{c,t} \leq  pStorageCap_c \quad \forall c \in C, t \in T
$$


In [8]:
@constraint(bio_model, cConvCap[t in T], 
    sum(vCONS[c,t] for c in C) <= pFacilityCap 
    )

    @constraint(bio_model, cStorageCap[c in C, t in T], 
    vSTOR[c,t] <= pStorageCap[c]
    )

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:2
    Dimension 2, 1:3
And data, a 2×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cStorageCap[1,1] : vSTOR[1,1] <= 30  …  cStorageCap[1,3] : vSTOR[1,3] <= 30
 cStorageCap[2,1] : vSTOR[2,1] <= 15     cStorageCap[2,3] : vSTOR[2,3] <= 15

3. Storage inventory balance

 The inventory of biomass in storage at the **end** of time period `t` is constrainted to be equal to the inventory at the **end** of the prior time period `t-1` plus the amount of biomass purchased in period `t` minus the biomass consumed in period `t`

 $$
vSTOR_{c,t} = vSTOR_{c,t-1} + vPUR_{c,t} -vCONS_{c,t} \quad \forall c \in C, t \in T
 $$ 

 This is an example of a recursive or **inter-temporal** constraint: the inventory in period `t` depends on inventory in prior period `t-1`. So we have to define a special case for the first time period. They are multiple ways to do this:
 -  define the  starting inventory, at the beginning of period `1` as the simulation defined by a parameter earlier.  
 - define the starting inventory at the beginning of period `1` to be equal to the inventory at the end of period `3`. This represents a so-called **cyclic constraints** reflecting the condition that the system reaches the original state at the end of the simulation. We will use this more nuanced constraint in the model but you can test the sensitivity of this assumption on your own.

 In the implementation below, we do this in a single line using the symbol `?` within constraint that is short for an `if-else`  condition

In [9]:
unregister(bio_model, :cStorSoI)
@constraint(bio_model, cStorSoI[c in C, t in T], 
    vSTOR[c,t] == (t == 1 ? vSTOR[c,T[end]] : vSTOR[c,t-1]) + vPUR[c,t] - vCONS[c,t]
    );

In [10]:
@show(cStorSoI[1,1])

cStorSoI[1, 1] = cStorSoI[1,1] : vCONS[1,1] - vPUR[1,1] + vSTOR[1,1] - vSTOR[1,3] == 0


cStorSoI[1,1] : vCONS[1,1] - vPUR[1,1] + vSTOR[1,1] - vSTOR[1,3] == 0

4. We need to constrain the biomass consumption in each period to be less than or equal to the available supply. This can be written as:

$$
vCONS_{c,t} \leq pSupply_{c,t} \quad \forall c \in C, t \in T
$$

5. The minimum energy constraint forces the plant to produce at least some amount in  each period, defined in this case in terms of input biomass processing capacity; This type of constraint may be necessary for practical reasons (e.g. cannot mothball production since it startup maybe expensive). This will however require using expensive feedstock or draining inventories.

In [11]:
@constraint(bio_model, conSupply[c in C, t in T], 
    vPUR[c,t] <= pSupply[t,c] );
    

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:2
    Dimension 2, 1:3
And data, a 2×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 conSupply[1,1] : vPUR[1,1] <= 20  …  conSupply[1,3] : vPUR[1,3] <= 15
 conSupply[2,1] : vPUR[2,1] <= 5      conSupply[2,3] : vPUR[2,3] <= 10

In [12]:
@constraint(bio_model, conMinEnergy[t in T], 
    sum(vCONS[c,t] for c in C) >= pFacilityMinInput  # minimum energy output of 50 GJ
    );

### 6. Model objective function 

We can now define the objective function of the optimization model, which is the operating profit for the facility, defined as sum of revenues (first term) minus operating costs (2nd and 3rd terms)

$$
Max \sum_{t\in T} {eREV_t -eOPEX_t}
$$

NOTE: In the implementation we have scaled all the biomass flows and capacity variables in the units of $10^5$ tonnes so the units of the objective function of $10^5\$$

In [13]:
@objective(bio_model, Max, sum(eREV[t] - eOPEX[t] for t in T) );

Let's take a look at our full model...


In [14]:
print(bio_model)

Max 36 vCONS[1,1] + 60 vCONS[2,1] - 30 vPUR[1,1] - vSTOR[1,1] - 50 vPUR[2,1] - 1.5 vSTOR[2,1] + 31.200000000000003 vCONS[1,2] + 52 vCONS[2,2] - 30 vPUR[1,2] - vSTOR[1,2] - 50 vPUR[2,2] - 1.5 vSTOR[2,2] + 38.400000000000006 vCONS[1,3] + 64 vCONS[2,3] - 30 vPUR[1,3] - vSTOR[1,3] - 50 vPUR[2,3] - 1.5 vSTOR[2,3]
Subject to
 cStorSoI[1,1] : vCONS[1,1] - vPUR[1,1] + vSTOR[1,1] - vSTOR[1,3] == 0
 cStorSoI[2,1] : vCONS[2,1] - vPUR[2,1] + vSTOR[2,1] - vSTOR[2,3] == 0
 cStorSoI[1,2] : vCONS[1,2] - vPUR[1,2] - vSTOR[1,1] + vSTOR[1,2] == 0
 cStorSoI[2,2] : vCONS[2,2] - vPUR[2,2] - vSTOR[2,1] + vSTOR[2,2] == 0
 cStorSoI[1,3] : vCONS[1,3] - vPUR[1,3] - vSTOR[1,2] + vSTOR[1,3] == 0
 cStorSoI[2,3] : vCONS[2,3] - vPUR[2,3] - vSTOR[2,2] + vSTOR[2,3] == 0
 conMinEnergy[1] : vCONS[1,1] + vCONS[2,1] >= 8
 conMinEnergy[2] : vCONS[1,2] + vCONS[2,2] >= 8
 conMinEnergy[3] : vCONS[1,3] + vCONS[2,3] >= 8
 cConvCap[1] : vCONS[1,1] + vCONS[2,1] <= 20
 cConvCap[2] : vCONS[1,2] + vCONS[2,2] <= 20
 cConvCap[3] : vCON


A little more complicated, eh? Now you can see why it is useful to have an algebraic programming language to define more complex models and then hand over to a solver to do the hard math. 

Lets optimize this problem!

In [15]:
optimize!(bio_model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 24 rows; 18 cols; 48 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+00, 3e+01]
Presolving model
12 rows, 18 cols, 36 nonzeros  0s
Dependent equations search running on 6 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
9 rows, 18 cols, 30 nonzeros  0s
Presolve : Reductions: rows 9(-15); columns 18(-0); elements 30(-18)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.8158477575e+02 Ph1: 9(12); Du: 6(281.585) 0s
         11    -4.2200000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 11
Objective value     :  4.2200000000e+02
P-D objective error :  0.0000000000e+00
HiGHS run ti

### Results!

We can see from the solver that the optimal operating profie is $4.22 \times 10^2 \times 10^5 \$$ 

We can now access the optimal decision variables as well. As these are now arrays of variables, we must use the [vectorized syntax](https://docs.julialang.org/en/v1/manual/functions/#man-vectorized) version of JuMP's `value()` function: `value.(ArrayOfVariablesReference)` 

Similarly, for or arrays of expressions or constraints, use: `value.(ArrayOfExpressionsReference)` or `value.(ArrayOfConstraintsReference)`.

For example, here's the optimal inventory of each biomass type across the three model years:

In [16]:
value.(vPUR)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:2
    Dimension 2, 1:3
And data, a 2×3 Matrix{Float64}:
 20.0  10.0   0.0
  5.0  12.0  10.0

In [17]:
value.(vCONS)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:2
    Dimension 2, 1:3
And data, a 2×3 Matrix{Float64}:
 15.0  15.0   0.0
  5.0   2.0  20.0

In [18]:
value.(vSTOR)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:2
    Dimension 2, 1:3
And data, a 2×3 Matrix{Float64}:
 5.0   0.0  -0.0
 0.0  10.0   0.0

Note that `value.(Reference)` will return a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/dev/variables/#variable_jump_arrays-1) which is not a data format that is compatible with DataFrames. To access the numerical values for an array of JuMP variables or expressions in a DenseAxisArray, you use the syntax `value.(Reference).data`.

We see that optimal solution involves buying more of the cheaper biomass (type 1) when its available, storing it,  and using it to meet the minimum input requirement when the expensive biomass (type 2) is the only one available.

Below we compile the DataFrame and write it to .csv file. Note that we copy in the time intervals and price in each time period from the input time series as well for context.


In [ ]:
results = DataFrame(
    Time = repeat(T, inner=length(C)),
    BiomassType = repeat(C, outer=length(T)),
    Consumption = vec(value.(vCONS).data),
    Purchase = vec(value.(vPUR).data),
    Storage = vec(value.(vSTOR).data)
)


Row,Time,BiomassType,Consumption,Purchase,Storage
,Int64,Int64,Float64,Float64,Float64
1,1,1,15.0,20.0,5.0
2,1,2,5.0,5.0,0.0
3,2,1,15.0,10.0,0.0
4,2,2,2.0,12.0,10.0
5,3,1,0.0,0.0,-0.0
6,3,2,20.0,10.0,0.0


In [21]:
# Let's create a folder for our results
path = "results" # Replace this with a different path if you prefer to write to another location
if !ispath(path) 
    mkdir(path)
end
write_path = joinpath(path, "biomass_supply_optimization_results.csv") 
CSV.write(write_path, results)

"results\\biomass_supply_optimization_results.csv"